In [1]:
import fastf1
import pandas as pd
import time

In [2]:
# Get historical data. Doesn't include 2025.
year = 2022
meetings_df = pd.DataFrame()
while year<2025:
    meeting = fastf1.get_event_schedule(year)
    meeting = meeting.loc[:,["RoundNumber","Country","Location","EventFormat"]]
    meeting['Year'] = year
    meetings_df = pd.concat([meetings_df,meeting],axis=0)
    year += 1

condition = meetings_df["EventFormat"] == "testing"
meetings_df = meetings_df[~condition]


req         WARNING 	DEFAULT CACHE ENABLED! (11.53 GB) C:\Users\rohan\AppData\Local\Temp\fastf1


In [3]:
meetings_df

,RoundNumber,Country,Location,EventFormat,Year
2,1,Bahrain,Sakhir,conventional,2022
3,2,Saudi Arabia,Jeddah,conventional,2022
4,3,Australia,Melbourne,conventional,2022
5,4,Italy,Imola,sprint,2022
6,5,United States,Miami,conventional,2022
...,...,...,...,...,...
20,20,Mexico,Mexico City,conventional,2024
21,21,Brazil,São Paulo,sprint_qualifying,2024
22,22,United States,Las Vegas,conventional,2024
23,23,Qatar,Lusail,sprint_qualifying,2024


In [209]:
#For EventFormat == sprint_shootout or EventFormat == sprint_qualifying

# For Qualifying 
# data from session.results
qual_results_df = qual.results.loc[:,["DriverNumber","Position"]]
qual_results_df = qual_results_df.rename(columns={"Position":"Qual_Position"})

#For Race 
#data from session.results
race_results_df = race.results.loc[:,["DriverNumber","BroadcastName","TeamName","Position"]]

# data from session.laps
race_laps_df = race.laps.loc[:,["DriverNumber","Sector1Time","Sector2Time","Sector3Time","SpeedST","Stint"]]
for sector in ['Sector1Time', 'Sector2Time', 'Sector3Time']: race_laps_df[sector] = race_laps_df[sector] / pd.Timedelta(seconds=1)
final_race_laps_df = race_laps_df.groupby('DriverNumber').agg({
    'Sector1Time': 'mean',
    'Sector2Time': 'mean',
    'Sector3Time': 'mean',
    'SpeedST': 'mean',
    'Stint': 'max'
}).reset_index()

# outer join dfs --> complete data for a single race session 
merged_race_df = pd.merge(race_results_df,final_race_laps_df,on="DriverNumber",how="outer")

# Merge qual and race data 
merged_qual_race_df = pd.merge(qual_results_df,merged_race_df,on="DriverNumber",how="outer")
merged_qual_race_df = merged_qual_race_df.rename(columns={"Position":"Race_Position"})

# Get Sprint data
sprint_results_df = sprint.results.loc[:,["DriverNumber","GridPosition","Position"]]
sprint_results_df = sprint_results_df.rename(columns={"GridPosition":"Sprint_Qual_Position","Position":"Sprint_Race_Position"})

# Merge Qual + Sprint + Race data
merged_qual_race_df = pd.merge(merged_qual_race_df,sprint_results_df,on="DriverNumber",how="outer")

merged_qual_race_df["Event_Type"] = "Sprint"

In [175]:
# For EventFormat == sprint

## note its "S" for sprint

# For Qualifying 
# data from session.results
qual_results_df = qual.results.loc[:,["DriverNumber","Position"]]
qual_results_df = qual_results_df.rename(columns={"Position":"Qual_Position"})

# For Race
#data from session.results
race_results_df = race.results.loc[:,["DriverNumber","BroadcastName","TeamName","Position"]]

# data from session.laps
race_laps_df = race.laps.loc[:,["DriverNumber","Sector1Time","Sector2Time","Sector3Time","SpeedST","Stint"]]
for sector in ['Sector1Time', 'Sector2Time', 'Sector3Time']: race_laps_df[sector] = race_laps_df[sector] / pd.Timedelta(seconds=1)
final_race_laps_df = race_laps_df.groupby('DriverNumber').agg({
    'Sector1Time': 'mean',
    'Sector2Time': 'mean',
    'Sector3Time': 'mean',
    'SpeedST': 'mean',
    'Stint': 'max'
}).reset_index()

# outer join dfs --> complete data for a single race session 
merged_race_df = pd.merge(race_results_df,final_race_laps_df,on="DriverNumber",how="outer")

# Merge qual and race data 
merged_qual_race_df = pd.merge(qual_results_df,merged_race_df,on="DriverNumber",how="outer")
merged_qual_race_df = merged_qual_race_df.rename(columns={"Position":"Race_Position"})

# Get Sprint data
sprint_results_df = sprint.results.loc[:,["DriverNumber","Position"]]
sprint_results_df = sprint_results_df.rename(columns={"Position":"Sprint_Race_Position"})

# Merge Qual + Sprint + Race data
merged_qual_race_df["Sprint_Qual_Position"] = merged_qual_race_df["Qual_Position"]
merged_qual_race_df = pd.merge(merged_qual_race_df,sprint_results_df,on="DriverNumber",how="outer")

merged_qual_race_df["Event_Type"] = "Sprint"


In [35]:
#For EventFormat == conventional

# For Qualifying 
# data from session.results
qual_results_df = qual.results.loc[:,["DriverNumber","Position"]]
qual_results_df = qual_results_df.rename(columns={"Position":"Qual_Position"})

# For Race
#data from session.results
race_results_df = race.results.loc[:,["DriverNumber","BroadcastName","TeamName","Position"]]

# data from session.laps
race_laps_df = race.laps.loc[:,["DriverNumber","Sector1Time","Sector2Time","Sector3Time","LapTime","SpeedST","Stint"]]
for sector in ['LapTime','Sector1Time', 'Sector2Time', 'Sector3Time']: race_laps_df[sector] = race_laps_df[sector] / pd.Timedelta(seconds=1)
final_race_laps_df = race_laps_df.groupby('DriverNumber').agg({
    'Sector1Time': 'mean',
    'Sector2Time': 'mean',
    'Sector3Time': 'mean',
    'LapTime': 'mean',
    'SpeedST': 'mean',
    'Stint': 'max'
}).reset_index()

# outer join dfs --> complete data for a single race session 
merged_race_df = pd.merge(race_results_df,final_race_laps_df,on="DriverNumber",how="outer")

# Merge qual and race data 
merged_qual_race_df = pd.merge(qual_results_df,merged_race_df,on="DriverNumber",how="outer")
merged_qual_race_df = merged_qual_race_df.rename(columns={"Position":"Race_Position"})

merged_qual_race_df["Sprint_Qual_Position"] = None
merged_qual_race_df["Sprint_Race_Position"] = None
merged_qual_race_df["Event_Type"] = "Race"

In [4]:
def get_event_info(year,round_num,event_format):
    
    qual = fastf1.get_session(year,round_num,'Q')
    qual.load()
    race = fastf1.get_session(year,round_num,'R')
    race.load()

    if event_format != "conventional":
        sprint = fastf1.get_session(year,round_num,'S')
        sprint.load()

    # For Qualifying 
    # data from session.results
    qual_results_df = qual.results.loc[:,["DriverNumber","Position"]]
    qual_results_df = qual_results_df.rename(columns={"Position":"Qual_Position"})

    # For Race
    # data from session.results
    race_results_df = race.results.loc[:,["DriverNumber","BroadcastName","TeamName","Position"]]

    # data from session.laps
    race_laps_df = race.laps.loc[:,["DriverNumber","Sector1Time","Sector2Time","Sector3Time","SpeedST","Stint"]]
    for sector in ['Sector1Time', 'Sector2Time', 'Sector3Time']: race_laps_df[sector] = race_laps_df[sector] / pd.Timedelta(seconds=1)
    final_race_laps_df = race_laps_df.groupby('DriverNumber').agg({
        'Sector1Time': 'mean',
        'Sector2Time': 'mean',
        'Sector3Time': 'mean',
        'SpeedST': 'mean',
        'Stint': 'max'
    }).reset_index()

    # outer join dfs --> complete data for a single race session 
    merged_race_df = pd.merge(race_results_df,final_race_laps_df,on="DriverNumber",how="outer")

    # Merge qual and race data 
    merged_qual_race_df = pd.merge(qual_results_df,merged_race_df,on="DriverNumber",how="outer")
    merged_qual_race_df = merged_qual_race_df.rename(columns={"Position":"Race_Position"})

    if event_format == "conventional":
        merged_qual_race_df["Sprint_Qual_Position"] = None
        merged_qual_race_df["Sprint_Race_Position"] = None
        merged_qual_race_df["Event_Type"] = "Race"

    elif event_format == "sprint":
        # Get Sprint data
        sprint_results_df = sprint.results.loc[:,["DriverNumber","Position"]]
        sprint_results_df = sprint_results_df.rename(columns={"Position":"Sprint_Race_Position"})

        # Merge Qual + Sprint + Race data
        merged_qual_race_df["Sprint_Qual_Position"] = merged_qual_race_df["Qual_Position"]
        merged_qual_race_df = pd.merge(merged_qual_race_df,sprint_results_df,on="DriverNumber",how="outer")

        merged_qual_race_df["Event_Type"] = "Sprint"

    elif event_format == "sprint_shootout" or event_format == "sprint_qualifying":
        # Get Sprint data
        sprint_results_df = sprint.results.loc[:,["DriverNumber","GridPosition","Position"]]
        sprint_results_df = sprint_results_df.rename(columns={"GridPosition":"Sprint_Qual_Position","Position":"Sprint_Race_Position"})

        # Merge Qual + Sprint + Race data
        merged_qual_race_df = pd.merge(merged_qual_race_df,sprint_results_df,on="DriverNumber",how="outer")

        merged_qual_race_df["Event_Type"] = "Sprint"

    else:
        raise Exception(f"Unknown event format: {event_format}")
    
    return merged_qual_race_df

In [5]:
just2022 = meetings_df.loc[meetings_df["Year"]==2022,:]

In [6]:
just2022

,RoundNumber,Country,Location,EventFormat,Year
2,1,Bahrain,Sakhir,conventional,2022
3,2,Saudi Arabia,Jeddah,conventional,2022
4,3,Australia,Melbourne,conventional,2022
5,4,Italy,Imola,sprint,2022
6,5,United States,Miami,conventional,2022
7,6,Spain,Barcelona,conventional,2022
8,7,Monaco,Monaco,conventional,2022
9,8,Azerbaijan,Baku,conventional,2022
10,9,Canada,Montréal,conventional,2022
11,10,Great Britain,Silverstone,conventional,2022


In [7]:
def get_data():  
    final_df = pd.DataFrame()
    for index,row in just2022.iterrows():
        round_num = row["RoundNumber"]
        country = row["Country"]
        location = row["Location"]
        EventFormat = row["EventFormat"]
        year = row["Year"]

        print(f"{index}: {round_num}, {country}, {location}, {EventFormat}, {year}")
        #time.sleep(5)
        
        meeting_data_df = get_event_info(year,round_num,EventFormat)

        meeting_data_df["Round_Number"] = round_num
        meeting_data_df["Country"] = country
        meeting_data_df["Location"] = location
        meeting_data_df["Year"] = year

        # reorganize columns
        new_col_order = ["DriverNumber","BroadcastName","TeamName","Round_Number","Country","Location","Year","Event_Type",
                        "Sprint_Qual_Position","Sprint_Race_Position","Qual_Position","Sector1Time","Sector2Time","Sector3Time",
                        "SpeedST","Stint","Race_Position"]
        meeting_data_df = meeting_data_df[new_col_order]

        final_df = pd.concat([final_df,meeting_data_df],axis=0)

    return final_df

In [8]:
d = get_data()
d.to_csv("train_data_2022.csv")

core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


2: 1, Bahrain, Sakhir, conventional, 2022


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '1', '55', '11', '44', '77', '20', '14', '63', '10', '31', '47', '4', '23', '24', '22', '27', '3', '18', '6']
core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req       

3: 2, Saudi Arabia, Jeddah, conventional, 2022


core        WARNING 	Driver 55: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 18: Lap timing integrity check failed for 1 lap(s)
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['11', '16', '55', '1', '31', '63', '14', '77', '10', '20', '4', '3', '24', '47', '18', '44', '23', '27', '6', '22']
core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req 

...

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '63', '44', '11', '55', '77', '16', '4', '14', '31', '3', '24', '22', '10', '20', '47', '5', '18', '23', '6']
core           INFO 	Loading data for Mexico City Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_co

22: 21, Brazil, São Paulo, sprint, 2022


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['20', '1', '63', '4', '55', '31', '14', '44', '11', '16', '23', '10', '5', '3', '18', '6', '24', '77', '22', '47']
core           INFO 	Loading data for São Paulo Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_coun

23: 22, Abu Dhabi, Yas Island, conventional, 2022


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '16', '55', '44', '63', '4', '31', '5', '3', '14', '22', '47', '18', '24', '20', '10', '77', '23', '6']
core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_coun

In [9]:
d

,DriverNumber,BroadcastName,TeamName,Round_Number,Country,Location,Year,Event_Type,Sprint_Qual_Position,Sprint_Race_Position,Qual_Position,Sector1Time,Sector2Time,Sector3Time,SpeedST,Stint,Race_Position
0,1,M VERSTAPPEN,Red Bull Racing,1,Bahrain,Sakhir,2022,Race,NaN,NaN,2.0,33.720943,44.042333,25.901444,278.895833,4.0,19.0
1,10,P GASLY,AlphaTauri,1,Bahrain,Sakhir,2022,Race,NaN,NaN,10.0,32.808465,43.315864,24.596659,291.687500,3.0,20.0
2,11,S PEREZ,Red Bull Racing,1,Bahrain,Sakhir,2022,Race,NaN,NaN,4.0,33.359873,44.062893,25.503964,282.716981,4.0,18.0
3,14,F ALONSO,Alpine,1,Bahrain,Sakhir,2022,Race,NaN,NaN,8.0,33.591357,44.109825,25.341860,284.638298,4.0,9.0
4,16,C LECLERC,Ferrari,1,Bahrain,Sakhir,2022,Race,NaN,NaN,1.0,33.484018,43.873719,25.345807,277.478261,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,5,S VETTEL,Aston Martin,22,Abu Dhabi,Yas Island,2022,Race,NaN,NaN,9.0,18.873421,38.982328,34.313897,303.724138,2.0,10.0
16,55,C SAINZ,Ferrari,22,Abu Dhabi,Yas Island,2022,Race,NaN,NaN,4.0,18.980421,38.700190,33.491000,303.775862,3.0,4.0
17,6,N LATIFI,Williams,22,Abu Dhabi,Yas Island,2022,Race,NaN,NaN,20.0,19.547019,39.537655,35.277327,299.600000,3.0,19.0
18,63,G RUSSELL,Mercedes,22,Abu Dhabi,Yas Island,2022,Race,NaN,NaN,6.0,19.113509,38.862724,33.375310,296.275862,3.0,5.0


In [35]:
just2024 = meetings_df.loc[meetings_df["Year"]==2024,:]

In [61]:
#qual = fastf1.get_session(2024,24,'Q')
#qual.load()
#race = fastf1.get_session(2024,24,'R')
#race.load()
#sprint = fastf1.get_session(2024,24,'S')
#sprint.load()

In [43]:
count = 0
bad = []
for _,row in just2024.iterrows():
        time.sleep(5)
        round_num = row["RoundNumber"]
        country = row["Country"]
        location = row["Location"]
        EventFormat = row["EventFormat"]
        year = row["Year"]
        
        if round_num >= 5:
                print(f"{count}: {round_num}, {country}, {location}, {EventFormat}, {year}")

                qual = fastf1.get_session(year,round_num,'Q')
                qual.load()
                race = fastf1.get_session(year,round_num,'R')
                race.load()

                if EventFormat != "conventional":
                        sprint = fastf1.get_session(year,round_num,'S')
                        sprint.load()

                if len(race.results) == 0 or len(qual.results)==0 or(EventFormat != "conventional" and len(sprint.results)==0):
                        bad.append(f"{round_num}-{location}-{sprint}-{year}")

        count += 1
        

4: 5, China, Shanghai, sprint_qualifying, 2024


core           INFO 	Loading data for Chinese Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	No cached data found for weather_data. Loading data...
_api           INFO 	Fetching weather data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for race_control_messages. Loading data...
_api           INFO 	Fetching race control messages...
req            INFO 	Data has been written to cache!
core           INFO 	Finished loading 

5: 6, United States, Miami, sprint_qualifying, 2024


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '55', '11', '4', '81', '63', '44', '27', '22', '18', '10', '31', '23', '14', '77', '2', '3', '20', '24']
core           INFO 	Loading data for Miami Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
r

6: 7, Italy, Imola, conventional, 2024


...

req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to

22: 23, Qatar, Lusail, sprint_qualifying, 2024


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to

23: 24, United Arab Emirates, Yas Island, conventional, 2024


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
logger      WARNING 	Failed to load timing da